In [105]:
import matplotlib.pyplot as plt
%matplotlib tk
import numpy as np

import torch
import torch.optim as optim
import torchvision.datasets as dataset
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

In [106]:
#### Classifying point in and out of circle ####


# Random points of {0,1}x{0,1}
number_of_points=1300
input_dimension=2
output_dimension=1

# Generate and classify points that are in (1) or out (-1) of the circle
x=np.random.rand(number_of_points,input_dimension)
y=np.array([1 if (i[0]-0.5)**2+(i[1]-0.5)**2<(0.3)**2 else 0 for i in x])
#y=np.array([1 if (i[0]-0.5)**2+(i[1]-0.5)**2<(0.2)**2 else 0 for i in x])


# Convert to pytorch tennsors

x_tensor=torch.from_numpy(x).type(torch.FloatTensor)
y_tensor=torch.from_numpy(y).type(torch.LongTensor)

# Convert to pytorch variables
x_variable=Variable(x_tensor,requires_grad=False)
y_variable=Variable(y_tensor,requires_grad=False)


In [107]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.fc2 = torch.nn.Linear(n_hidden, n_hidden)   # hidden layer

        self.fc3 = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.fc1(x))      # activation function for hidden layer
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))

        return x


In [108]:
# Set hyperparameters #
net = Net(n_feature=2, n_hidden=10, n_output=2)     # define the network
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.0004,momentum=0.9)
loss_func = torch.nn.CrossEntropyLoss()  # the target label is NOT an one-hotte



Net(
  (fc1): Linear(in_features=2, out_features=10)
  (fc2): Linear(in_features=10, out_features=10)
  (fc3): Linear(in_features=10, out_features=2)
)


In [111]:
nb_of_epochs=10000
batch_size=100
# Train the network #
for t in range(nb_of_epochs):
    sum_loss=0
    for b in range(0,x_variable.size(0),batch_size):
        
        out = net(x_variable.narrow(0,b,batch_size))                 # input x and predict based on x
        loss = loss_func(out, y_variable.narrow(0,b,batch_size))     # must be (1. nn output, 2. target), the target label is NOT one-hotted
        sum_loss+=loss.data[0]
        optimizer.zero_grad()   # clear gradients for next train
        loss.backward()         # backpropagation, compute gradients
        #print(t,loss.data[0])
        optimizer.step()        # apply gradients
    print("Epoch:",t,"Loss:",sum_loss)

Epoch: 0 Loss: 0.5401488170027733
Epoch: 1 Loss: 0.5401264280080795
Epoch: 2 Loss: 0.5401040744036436
Epoch: 3 Loss: 0.5400816928595304
Epoch: 4 Loss: 0.5400591045618057
Epoch: 5 Loss: 0.5400370322167873
Epoch: 6 Loss: 0.5400146264582872
Epoch: 7 Loss: 0.5399922467768192
Epoch: 8 Loss: 0.5399699229747057
Epoch: 9 Loss: 0.5399473961442709
Epoch: 10 Loss: 0.5399253685027361
Epoch: 11 Loss: 0.5399027448147535
Epoch: 12 Loss: 0.5398803427815437
Epoch: 13 Loss: 0.5398582443594933
Epoch: 14 Loss: 0.5398357026278973
Epoch: 15 Loss: 0.5398134272545576
Epoch: 16 Loss: 0.5397912785410881
Epoch: 17 Loss: 0.5397688094526529
Epoch: 18 Loss: 0.5397465508431196
Epoch: 19 Loss: 0.5397242903709412
Epoch: 20 Loss: 0.5397020801901817
Epoch: 21 Loss: 0.5396797321736813
Epoch: 22 Loss: 0.5396575331687927
Epoch: 23 Loss: 0.5396350268274546
Epoch: 24 Loss: 0.5396128948777914
Epoch: 25 Loss: 0.53959053568542
Epoch: 26 Loss: 0.5395682640373707
Epoch: 27 Loss: 0.539546050131321
Epoch: 28 Loss: 0.539523864164948

In [110]:
# Creating test data to check validity #
x_test_array=np.random.rand(4000,input_dimension)

x_test=Variable(torch.from_numpy(x_test_array).type(torch.FloatTensor))
y_pred=net(x_test)
y=[0 if i>0 else 1 for i,j in y_pred.data.numpy()]
col=['r' if i==0 else 'b' for i in y]
plt.scatter(x_test_array[:,0],x_test_array[:,1],c=col)
